In [1]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np

import pandas as pd
import torch
import torch.nn as nn

In [2]:
df=pd.read_csv("diabetes.csv",header=None)

In [3]:
df.values

array([[-0.294118 ,  0.487437 ,  0.180328 , ..., -0.53117  , -0.0333333,
         0.       ],
       [-0.882353 , -0.145729 ,  0.0819672, ..., -0.766866 , -0.666667 ,
         1.       ],
       [-0.0588235,  0.839196 ,  0.0491803, ..., -0.492741 , -0.633333 ,
         0.       ],
       ...,
       [-0.411765 ,  0.21608  ,  0.180328 , ..., -0.857387 , -0.7      ,
         1.       ],
       [-0.882353 ,  0.266332 , -0.0163934, ..., -0.768574 , -0.133333 ,
         0.       ],
       [-0.882353 , -0.0653266,  0.147541 , ..., -0.797609 , -0.933333 ,
         1.       ]])

# Experimentare cu diferiti parametrii

In toate experimentele am tinut fixe toti parametrii in afara de cei testati. Configuratia de baza este

Batch size 16

Epochs 200

lr 0.08

Layers: self.sequential=nn.Sequential(
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,1),
            nn.Sigmoid()
        )

## 1. Batch size

a. 16 -> loss Mean 6.8248950988054276 -> mean loss/input 0.4275

b. 32 -> loss Mean 13.75011432170868 -> mean loss/input  0.4281

c. 128 -> loss Mean 58.93515205383301 -> mean loss/input 0.4603

Concluzie: Cu cat batch size-ul este mai mare cu atat modelul de comporta mai rau si mai incet. Asta se intampla deoarece nu invata din fiecare exemplu si incearca sa invete din mai multe o data. Batch-size mare trebuie folosit cand sunt foarte multe date pentru a creste viteza de procesare. Cu toate actestea, cu cat batch-ul este mai mare cu atat modelul se comporta mai slab


## 2. Numar de neuroni / layer

Am folosit urmatoarea structura:

self.sequential=nn.Sequential(
            nn.Linear(8,120),
            nn.Sigmoid(),
            nn.Linear(120,180),
            nn.Sigmoid(),
            nn.Linear(180,1),
            nn.Sigmoid()
        )
        
Mean 7.409046034018199 -> mean loss/input 0.4625

Concluzie: Rezultatul este mai rau din cauza ca functia este foarte complicata si nu reuseste sa generalizeze bine sau nu este potrivita pentru setul de date.

## 3. Numar de layere

Am folosit urmatoarea structura:

self.sequential=nn.Sequential(
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8, 12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,1),
            nn.Sigmoid()
        )

Mean 10.310399025678635 -> mean loss/input 0.6444

Concluzie: Algoritmul nu converge sau converge foarte greu. Cu cat este mai lunga reteaua cu atat ii este mai greu sa modifice bine weight-urile deoarece fiecare weight are un impact mic, si atunci derivatele sunt mici si modificarea este mica. 

## 4. Learning rate

a. learning rate 0.15

Mean 6.653563092152278 -> mean loss/input 0.4158125

b. learning rate 0.6

Mean 9.322209392984709 -> mean loss/input 0.5825 + faptul ca algoritmul nu converge

c. learning rate 0.08

Mean 6.8248950988054276 -> mean loss/input 0.4275

Concluzie: learning rate-ul face ca algoritmul sa convearga si cat de repede sa convearga. Chiar daca in urma rezultatealor de mai sus learning rate-ul potrivit pare a fii 0.15, in cazul in care se creste numarul de epochs, algoritmul nu mai converge de la un punct.

In [22]:
#Dataset - o clasă din PyTorch foarte utilă gestionării seturilor de date
class DiabetesDataset(Dataset):
    """ Diabetes dataset."""
    # Initialize your data, download, etc.
    def __init__(self):
        #Citim setul de date
        df=pd.read_csv("diabetes.csv",header=None, dtype=np.float32)
        xy = torch.from_numpy(df.values)
        self.len = xy.shape[0]
        #Vom folosi ca input toate valorile mai puțin ultima coloană
        self.x_data = xy[:, 0:-1]
        #Vom folosi ca output ultima coloană
        self.y_data = xy[:, [-1]]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [23]:
dataset = DiabetesDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=16,
                          shuffle=True,
                          num_workers=1)

In [18]:
dataset[0]
dataset.len

759

In [36]:

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        torch.manual_seed(0)
        #Practic toate functiile liniare și non-liniare realizează o secvență a flowlui de date
        self.sequential=nn.Sequential(
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.sequential(x)

In [28]:
class ModelLat(nn.Module):

    def __init__(self):
        super(ModelLat, self).__init__()
        torch.manual_seed(0)
        #Practic toate functiile liniare și non-liniare realizează o secvență a flowlui de date
        self.sequential=nn.Sequential(
            nn.Linear(8,120),
            nn.Sigmoid(),
            nn.Linear(120,180),
            nn.Sigmoid(),
            nn.Linear(180,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.sequential(x)

In [29]:
model=ModelLat()

In [32]:
class ModelLung(nn.Module):

    def __init__(self):
        """
        Dat fiind faptul că 
        """
        super(ModelLung, self).__init__()
        torch.manual_seed(0)
        #Practic toate functiile liniare și non-liniare realizează o secvență a flowlui de date
        self.sequential=nn.Sequential(
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8, 12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,12),
            nn.Sigmoid(),
            nn.Linear(12,8),
            nn.Sigmoid(),
            nn.Linear(8,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.sequential(x)

In [33]:
model=ModelLung()

In [40]:
model=Model()

In [41]:
criterion = nn.BCELoss(reduction='mean')
criterion2 = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.6)

In [42]:
# Training loop
for epoch in range(200):
  #Aplicăm modificarea de learning rate per epocă, deci va trebui să avem un istoric al loss-ului
  losses=[]
  for i, data in enumerate(train_loader, 0):
      # get the inputs
      inputs, labels = data

      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model(inputs)

      # Compute and print loss
      loss = criterion(y_pred, labels)
      loss2 = criterion2(y_pred, labels)  
      print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss2.item():.4f}')
      losses.append(loss2.item())
      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss2.backward()
      optimizer.step()
  #if epoch % 100 == 0:
  mean_loss=sum(losses)/len(losses)
  print(f'Mean {mean_loss}')  
  #scheduler.step(mean_loss)

Epoch 1 | Batch: 1 | Loss: 10.9848
Epoch 1 | Batch: 2 | Loss: 12.9700
Epoch 1 | Batch: 3 | Loss: 33.0138
Epoch 1 | Batch: 4 | Loss: 18.4174
Epoch 1 | Batch: 5 | Loss: 20.5828
Epoch 1 | Batch: 6 | Loss: 11.6737
Epoch 1 | Batch: 7 | Loss: 10.0719
Epoch 1 | Batch: 8 | Loss: 8.9911
Epoch 1 | Batch: 9 | Loss: 10.0674
Epoch 1 | Batch: 10 | Loss: 10.0695
Epoch 1 | Batch: 11 | Loss: 7.8711
Epoch 1 | Batch: 12 | Loss: 6.0832
Epoch 1 | Batch: 13 | Loss: 15.9492
Epoch 1 | Batch: 14 | Loss: 13.2772
Epoch 1 | Batch: 15 | Loss: 7.7508
Epoch 1 | Batch: 16 | Loss: 13.9231
Epoch 1 | Batch: 17 | Loss: 15.9064
Epoch 1 | Batch: 18 | Loss: 18.9490
Epoch 1 | Batch: 19 | Loss: 11.8343
Epoch 1 | Batch: 20 | Loss: 19.9313
Epoch 1 | Batch: 21 | Loss: 11.1460
Epoch 1 | Batch: 22 | Loss: 7.7663
Epoch 1 | Batch: 23 | Loss: 18.8790
Epoch 1 | Batch: 24 | Loss: 34.6083
Epoch 1 | Batch: 25 | Loss: 8.0873
Epoch 1 | Batch: 26 | Loss: 12.3708
Epoch 1 | Batch: 27 | Loss: 10.9619
Epoch 1 | Batch: 28 | Loss: 11.0670
Epoch 1

Epoch 6 | Batch: 1 | Loss: 9.5398
Epoch 6 | Batch: 2 | Loss: 15.4542
Epoch 6 | Batch: 3 | Loss: 16.9669
Epoch 6 | Batch: 4 | Loss: 26.8135
Epoch 6 | Batch: 5 | Loss: 11.3798
Epoch 6 | Batch: 6 | Loss: 7.6738
Epoch 6 | Batch: 7 | Loss: 12.8157
Epoch 6 | Batch: 8 | Loss: 18.3528
Epoch 6 | Batch: 9 | Loss: 17.7010
Epoch 6 | Batch: 10 | Loss: 9.4209
Epoch 6 | Batch: 11 | Loss: 10.2367
Epoch 6 | Batch: 12 | Loss: 10.7108
Epoch 6 | Batch: 13 | Loss: 7.7410
Epoch 6 | Batch: 14 | Loss: 5.1658
Epoch 6 | Batch: 15 | Loss: 6.3044
Epoch 6 | Batch: 16 | Loss: 7.8694
Epoch 6 | Batch: 17 | Loss: 9.5561
Epoch 6 | Batch: 18 | Loss: 17.8523
Epoch 6 | Batch: 19 | Loss: 12.2103
Epoch 6 | Batch: 20 | Loss: 7.3212
Epoch 6 | Batch: 21 | Loss: 4.8653
Epoch 6 | Batch: 22 | Loss: 6.6032
Epoch 6 | Batch: 23 | Loss: 8.9278
Epoch 6 | Batch: 24 | Loss: 11.7168
Epoch 6 | Batch: 25 | Loss: 4.5434
Epoch 6 | Batch: 26 | Loss: 6.7761
Epoch 6 | Batch: 27 | Loss: 11.4792
Epoch 6 | Batch: 28 | Loss: 6.3922
Epoch 6 | Batch:

Epoch 11 | Batch: 1 | Loss: 6.6072
Epoch 11 | Batch: 2 | Loss: 10.9744
Epoch 11 | Batch: 3 | Loss: 11.9749
Epoch 11 | Batch: 4 | Loss: 9.5216
Epoch 11 | Batch: 5 | Loss: 11.6638
Epoch 11 | Batch: 6 | Loss: 9.5180
Epoch 11 | Batch: 7 | Loss: 8.5855
Epoch 11 | Batch: 8 | Loss: 16.0211
Epoch 11 | Batch: 9 | Loss: 9.8917
Epoch 11 | Batch: 10 | Loss: 17.6865
Epoch 11 | Batch: 11 | Loss: 13.4545
Epoch 11 | Batch: 12 | Loss: 15.0469
Epoch 11 | Batch: 13 | Loss: 15.9126
Epoch 11 | Batch: 14 | Loss: 3.6043
Epoch 11 | Batch: 15 | Loss: 12.2523
Epoch 11 | Batch: 16 | Loss: 10.5818
Epoch 11 | Batch: 17 | Loss: 5.4113
Epoch 11 | Batch: 18 | Loss: 13.7526
Epoch 11 | Batch: 19 | Loss: 8.2559
Epoch 11 | Batch: 20 | Loss: 12.1638
Epoch 11 | Batch: 21 | Loss: 17.4502
Epoch 11 | Batch: 22 | Loss: 21.7202
Epoch 11 | Batch: 23 | Loss: 10.0723
Epoch 11 | Batch: 24 | Loss: 8.7377
Epoch 11 | Batch: 25 | Loss: 8.2770
Epoch 11 | Batch: 26 | Loss: 5.5737
Epoch 11 | Batch: 27 | Loss: 11.8854
Epoch 11 | Batch: 28 

Epoch 15 | Batch: 34 | Loss: 5.0823
Epoch 15 | Batch: 35 | Loss: 11.8118
Epoch 15 | Batch: 36 | Loss: 6.7169
Epoch 15 | Batch: 37 | Loss: 4.3385
Epoch 15 | Batch: 38 | Loss: 12.0389
Epoch 15 | Batch: 39 | Loss: 6.8147
Epoch 15 | Batch: 40 | Loss: 10.1074
Epoch 15 | Batch: 41 | Loss: 6.5383
Epoch 15 | Batch: 42 | Loss: 11.5123
Epoch 15 | Batch: 43 | Loss: 11.4545
Epoch 15 | Batch: 44 | Loss: 22.7542
Epoch 15 | Batch: 45 | Loss: 11.1149
Epoch 15 | Batch: 46 | Loss: 5.8525
Epoch 15 | Batch: 47 | Loss: 6.7053
Epoch 15 | Batch: 48 | Loss: 3.2885
Mean 9.733614186445871
Epoch 16 | Batch: 1 | Loss: 11.4019
Epoch 16 | Batch: 2 | Loss: 10.7662
Epoch 16 | Batch: 3 | Loss: 13.9804
Epoch 16 | Batch: 4 | Loss: 14.6747
Epoch 16 | Batch: 5 | Loss: 19.0618
Epoch 16 | Batch: 6 | Loss: 10.5511
Epoch 16 | Batch: 7 | Loss: 10.6504
Epoch 16 | Batch: 8 | Loss: 14.3417
Epoch 16 | Batch: 9 | Loss: 17.5759
Epoch 16 | Batch: 10 | Loss: 9.7348
Epoch 16 | Batch: 11 | Loss: 9.9143
Epoch 16 | Batch: 12 | Loss: 13.23

Epoch 20 | Batch: 22 | Loss: 16.5751
Epoch 20 | Batch: 23 | Loss: 31.0379
Epoch 20 | Batch: 24 | Loss: 25.3739
Epoch 20 | Batch: 25 | Loss: 7.8341
Epoch 20 | Batch: 26 | Loss: 13.7043
Epoch 20 | Batch: 27 | Loss: 18.6030
Epoch 20 | Batch: 28 | Loss: 23.8912
Epoch 20 | Batch: 29 | Loss: 11.3049
Epoch 20 | Batch: 30 | Loss: 11.1135
Epoch 20 | Batch: 31 | Loss: 6.2549
Epoch 20 | Batch: 32 | Loss: 7.1239
Epoch 20 | Batch: 33 | Loss: 5.5702
Epoch 20 | Batch: 34 | Loss: 12.3162
Epoch 20 | Batch: 35 | Loss: 6.4417
Epoch 20 | Batch: 36 | Loss: 13.3647
Epoch 20 | Batch: 37 | Loss: 8.0846
Epoch 20 | Batch: 38 | Loss: 2.2670
Epoch 20 | Batch: 39 | Loss: 13.5876
Epoch 20 | Batch: 40 | Loss: 21.2649
Epoch 20 | Batch: 41 | Loss: 13.0430
Epoch 20 | Batch: 42 | Loss: 9.6858
Epoch 20 | Batch: 43 | Loss: 8.1595
Epoch 20 | Batch: 44 | Loss: 9.6320
Epoch 20 | Batch: 45 | Loss: 20.7696
Epoch 20 | Batch: 46 | Loss: 18.3259
Epoch 20 | Batch: 47 | Loss: 10.3250
Epoch 20 | Batch: 48 | Loss: 6.6832
Mean 11.7637

...

Epoch 179 | Batch: 1 | Loss: 7.5186
Epoch 179 | Batch: 2 | Loss: 4.5457
Epoch 179 | Batch: 3 | Loss: 3.2373
Epoch 179 | Batch: 4 | Loss: 10.5212
Epoch 179 | Batch: 5 | Loss: 5.4758
Epoch 179 | Batch: 6 | Loss: 14.6122
Epoch 179 | Batch: 7 | Loss: 11.9798
Epoch 179 | Batch: 8 | Loss: 14.6282
Epoch 179 | Batch: 9 | Loss: 6.3080
Epoch 179 | Batch: 10 | Loss: 5.6667
Epoch 179 | Batch: 11 | Loss: 9.5941
Epoch 179 | Batch: 12 | Loss: 9.4950
Epoch 179 | Batch: 13 | Loss: 8.9385
Epoch 179 | Batch: 14 | Loss: 4.7429
Epoch 179 | Batch: 15 | Loss: 7.9098
Epoch 179 | Batch: 16 | Loss: 5.3408
Epoch 179 | Batch: 17 | Loss: 6.3127
Epoch 179 | Batch: 18 | Loss: 6.1515
Epoch 179 | Batch: 19 | Loss: 7.3220
Epoch 179 | Batch: 20 | Loss: 11.5330
Epoch 179 | Batch: 21 | Loss: 9.1973
Epoch 179 | Batch: 22 | Loss: 8.6942
Epoch 179 | Batch: 23 | Loss: 10.5458
Epoch 179 | Batch: 24 | Loss: 4.1114
Epoch 179 | Batch: 25 | Loss: 7.9410
Epoch 179 | Batch: 26 | Loss: 6.1427
Epoch 179 | Batch: 27 | Loss: 10.8684
Epo

Epoch 184 | Batch: 1 | Loss: 9.6932
Epoch 184 | Batch: 2 | Loss: 10.4881
Epoch 184 | Batch: 3 | Loss: 11.5545
Epoch 184 | Batch: 4 | Loss: 7.8730
Epoch 184 | Batch: 5 | Loss: 2.6755
Epoch 184 | Batch: 6 | Loss: 2.1662
Epoch 184 | Batch: 7 | Loss: 10.6206
Epoch 184 | Batch: 8 | Loss: 6.9326
Epoch 184 | Batch: 9 | Loss: 8.5917
Epoch 184 | Batch: 10 | Loss: 12.9557
Epoch 184 | Batch: 11 | Loss: 9.7530
Epoch 184 | Batch: 12 | Loss: 2.5871
Epoch 184 | Batch: 13 | Loss: 7.7558
Epoch 184 | Batch: 14 | Loss: 10.2166
Epoch 184 | Batch: 15 | Loss: 6.5391
Epoch 184 | Batch: 16 | Loss: 10.7676
Epoch 184 | Batch: 17 | Loss: 14.6115
Epoch 184 | Batch: 18 | Loss: 9.3304
Epoch 184 | Batch: 19 | Loss: 6.1559
Epoch 184 | Batch: 20 | Loss: 8.4232
Epoch 184 | Batch: 21 | Loss: 5.3204
Epoch 184 | Batch: 22 | Loss: 9.8231
Epoch 184 | Batch: 23 | Loss: 4.0679
Epoch 184 | Batch: 24 | Loss: 3.1684
Epoch 184 | Batch: 25 | Loss: 1.6226
Epoch 184 | Batch: 26 | Loss: 11.0160
Epoch 184 | Batch: 27 | Loss: 6.7855
Ep

Epoch 189 | Batch: 1 | Loss: 5.6512
Epoch 189 | Batch: 2 | Loss: 5.0860
Epoch 189 | Batch: 3 | Loss: 7.6842
Epoch 189 | Batch: 4 | Loss: 2.5011
Epoch 189 | Batch: 5 | Loss: 5.4830
Epoch 189 | Batch: 6 | Loss: 3.3482
Epoch 189 | Batch: 7 | Loss: 5.6757
Epoch 189 | Batch: 8 | Loss: 5.6246
Epoch 189 | Batch: 9 | Loss: 5.1315
Epoch 189 | Batch: 10 | Loss: 17.7856
Epoch 189 | Batch: 11 | Loss: 8.4377
Epoch 189 | Batch: 12 | Loss: 4.8871
Epoch 189 | Batch: 13 | Loss: 5.9681
Epoch 189 | Batch: 14 | Loss: 4.7890
Epoch 189 | Batch: 15 | Loss: 4.1557
Epoch 189 | Batch: 16 | Loss: 20.1080
Epoch 189 | Batch: 17 | Loss: 16.9276
Epoch 189 | Batch: 18 | Loss: 14.6306
Epoch 189 | Batch: 19 | Loss: 5.4368
Epoch 189 | Batch: 20 | Loss: 10.9248
Epoch 189 | Batch: 21 | Loss: 11.5441
Epoch 189 | Batch: 22 | Loss: 4.3063
Epoch 189 | Batch: 23 | Loss: 8.8117
Epoch 189 | Batch: 24 | Loss: 18.5930
Epoch 189 | Batch: 25 | Loss: 7.0023
Epoch 189 | Batch: 26 | Loss: 12.2638
Epoch 189 | Batch: 27 | Loss: 10.4261
E

Epoch 194 | Batch: 1 | Loss: 12.5380
Epoch 194 | Batch: 2 | Loss: 7.9996
Epoch 194 | Batch: 3 | Loss: 6.2459
Epoch 194 | Batch: 4 | Loss: 8.3607
Epoch 194 | Batch: 5 | Loss: 3.6367
Epoch 194 | Batch: 6 | Loss: 10.7117
Epoch 194 | Batch: 7 | Loss: 10.7791
Epoch 194 | Batch: 8 | Loss: 8.1854
Epoch 194 | Batch: 9 | Loss: 9.9519
Epoch 194 | Batch: 10 | Loss: 2.5024
Epoch 194 | Batch: 11 | Loss: 14.2809
Epoch 194 | Batch: 12 | Loss: 20.1460
Epoch 194 | Batch: 13 | Loss: 17.0444
Epoch 194 | Batch: 14 | Loss: 6.4358
Epoch 194 | Batch: 15 | Loss: 7.7265
Epoch 194 | Batch: 16 | Loss: 6.6166
Epoch 194 | Batch: 17 | Loss: 4.8672
Epoch 194 | Batch: 18 | Loss: 13.1170
Epoch 194 | Batch: 19 | Loss: 8.2111
Epoch 194 | Batch: 20 | Loss: 8.7972
Epoch 194 | Batch: 21 | Loss: 11.9370
Epoch 194 | Batch: 22 | Loss: 3.9600
Epoch 194 | Batch: 23 | Loss: 11.3576
Epoch 194 | Batch: 24 | Loss: 6.1026
Epoch 194 | Batch: 25 | Loss: 10.4797
Epoch 194 | Batch: 26 | Loss: 20.4670
Epoch 194 | Batch: 27 | Loss: 12.189

Epoch 198 | Batch: 43 | Loss: 6.8753
Epoch 198 | Batch: 44 | Loss: 12.3578
Epoch 198 | Batch: 45 | Loss: 15.6697
Epoch 198 | Batch: 46 | Loss: 8.2134
Epoch 198 | Batch: 47 | Loss: 13.3058
Epoch 198 | Batch: 48 | Loss: 9.4892
Mean 9.372293437520662
Epoch 199 | Batch: 1 | Loss: 8.4735
Epoch 199 | Batch: 2 | Loss: 5.8246
Epoch 199 | Batch: 3 | Loss: 8.5781
Epoch 199 | Batch: 4 | Loss: 6.1993
Epoch 199 | Batch: 5 | Loss: 1.9054
Epoch 199 | Batch: 6 | Loss: 5.8877
Epoch 199 | Batch: 7 | Loss: 6.8204
Epoch 199 | Batch: 8 | Loss: 6.6202
Epoch 199 | Batch: 9 | Loss: 11.2828
Epoch 199 | Batch: 10 | Loss: 17.1008
Epoch 199 | Batch: 11 | Loss: 36.7273
Epoch 199 | Batch: 12 | Loss: 13.8306
Epoch 199 | Batch: 13 | Loss: 22.0630
Epoch 199 | Batch: 14 | Loss: 20.7525
Epoch 199 | Batch: 15 | Loss: 16.8241
Epoch 199 | Batch: 16 | Loss: 10.8058
Epoch 199 | Batch: 17 | Loss: 10.4935
Epoch 199 | Batch: 18 | Loss: 10.9297
Epoch 199 | Batch: 19 | Loss: 9.1650
Epoch 199 | Batch: 20 | Loss: 7.9258
Epoch 199 |